In [1]:
import json
import os

import boto3
import requests
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from dotenv import load_dotenv
from rich.pretty import pprint

load_dotenv()

True

### Let's define the tools

In [2]:
check_weather_tool = {
    "name": "get_weather",
    "description": "Get the current weather in a given location",
    "input_schema": {
        "type": "object",
        "properties": {
            "latitude": {
                "type": "string",
                "description": "latitude of the location for which to get the weather",
            },
            "longitude": {
                "type": "string",
                "description": "longitude of the location for which to get the weather",
            },
        },
        "required": ["latitude", "longitude"],
    },
}

tools = [check_weather_tool]

### Let's make the first LLM call to decide which tool to use and with what parameters

In [3]:
region = os.getenv("AWS_REGION", "eu-central-1")
url = f"https://bedrock-runtime.{region}.amazonaws.com/model/eu.anthropic.claude-sonnet-4-5-20250929-v1:0/invoke"

llm_headers = {
    "Content-Type": "application/json",
}
session = boto3.Session()
credentials = session.get_credentials()

payload = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 300,
    "tools": tools,
    "tool_choice": {"type": "auto"},
    "messages": [
        {
            "role": "user",
            "content": "What's the weather in Warsaw? Is there anything interesting about this city?",
        }
    ]
}

request = AWSRequest(method='POST', url=url, data=json.dumps(payload), headers=llm_headers)
SigV4Auth(credentials, "bedrock", region).add_auth(request)

resp = requests.post(url, headers=dict(request.headers), data=json.dumps(payload))
response_content = resp.json().get("content", [])
resp.json()

{'model': 'claude-sonnet-4-5-20250929',
 'id': 'msg_bdrk_01RNkQ8wmw7JgogM9v9oaApg',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': "I'll help you get the weather for Warsaw and share some interesting facts about the city!"},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_01QUWmmD9kEJDr5V9SiDWP5V',
   'name': 'get_weather',
   'input': {'latitude': '52.2297', 'longitude': '21.0122'}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 617,
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'output_tokens': 95}}

### Now let's process the output from the tool use step

In [4]:
# Extracting tools
tools = [block for block in response_content if block.get("type") == "tool_use"]

In [ ]:
# Use tools
for tool in tools:
    if tool["name"] == "get_weather":
        latitude = tool["input"]["latitude"]
        longitude = tool["input"]["longitude"]
        weather_tool_response = requests.get(
            url="https://api.open-meteo.com/v1/forecast",
            params={"latitude": latitude, "longitude": longitude, "current_weather": True},
            verify=False,
            proxies={'http': '', 'https': ''},
        )
        weather_info = weather_tool_response.json()["current_weather"]
        tool["output"] = weather_info
    if tool["name"] == "facts_from_wikipedia":
        topic = tool["input"]["topic"]
        headers = {
            'User-Agent': 'Agent101Bot/1.0 (Educational purpose only)',
            'Accept': 'application/json'
        }
        wikipedia_response = requests.get(
            url=f"https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles={topic}&rvprop=content&format=json",
            headers=headers,
            verify=False,
            proxies={'http': '', 'https': ''},
        )
        wiki_info = wikipedia_response.json()
        tool["output"] = wiki_info

In [7]:
# Collecting outputs from all tools into a single string
tool_outputs = "\n".join([f"Tool: {tool['name']}, Output: {tool.get('output', 'No output')}" for tool in tools])
tool_outputs

"Tool: get_weather, Output: {'time': '2025-10-24T06:00', 'interval': 900, 'temperature': 12.1, 'windspeed': 19.4, 'winddirection': 294, 'is_day': 1, 'weathercode': 3}"